In [1]:
# data manipulation
import numpy as np
import pandas as pd

#file processing / requests
from pathlib import Path
from tqdm import tqdm
import json
import urllib

#computer vision
import cv2
import torch
import torchvision

import PIL

#ML
from sklearn.model_selection import train_test_split


#display
from IPython.display import display
from pylab import rcParams

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import PIL.Image as Image

#utils2.py package can be found under https://github.com/philippe-heitzmann/Object_Detection
import sys
sys.path.append('/Users/Administrator/DS/DeepRoad/Object_Detection')
sys.path.append('/Users/phil0/DS/DeepRoad/Object_Detection')
import utils2
from importlib import reload
reload(utils2)
from utils2 import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)

pd.set_option('display.float_format', lambda x: '%.2f' % x)

NOTE! Installing ujson may make loading annotations faster.


In [14]:
!pip freeze

absl-py==0.13.0
alabaster==0.7.12
apex==0.1
appdirs==1.4.4
argon2-cffi==21.1.0
asgiref==3.4.1
attrs==21.2.0
audioread==2.1.9
Babel==2.9.1
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_1601745390275/work
bleach==4.1.0
blis @ file:///home/conda/feedstock_root/build_artifacts/cython-blis_1607338147605/work
bokeh==2.4.2
brotlipy==0.7.0
cachetools==4.2.2
catalogue @ file:///home/conda/feedstock_root/build_artifacts/catalogue_1629144899387/work
certifi==2021.5.30
cffi @ file:///home/conda/feedstock_root/build_artifacts/cffi_1625835293160/work
chardet @ file:///home/conda/feedstock_root/build_artifacts/chardet_1610093490430/work
charset-normalizer @ file:///home/conda/feedstock_root/build_artifacts/charset-normalizer_1626371162869/work
click

In [5]:
# from https://github.com/CSAILVision/places365/blob/master/run_placesCNN_basic.py

# PlacesCNN for scene classification
#
# by Bolei Zhou
# last modified by Bolei Zhou, Dec.27, 2017 with latest pytorch and torchvision (upgrade your torchvision please if there is trn.Resize error)

import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image

# the architecture to use
arch = 'resnet18'

# load the pre-trained weights
model_file = '%s_places365.pth.tar' % arch
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

--2022-03-21 21:52:55--  http://places2.csail.mit.edu/models_places365/resnet18_places365.pth.tar
Resolving places2.csail.mit.edu (places2.csail.mit.edu)... 128.30.195.26, 128.30.195.26
Connecting to places2.csail.mit.edu (places2.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45506139 (43M) [application/x-tar]
Saving to: ‘resnet18_places365.pth.tar’

     0K .......... .......... .......... .......... ..........  0% 1.83M 24s
    50K .......... .......... .......... .......... ..........  0% 3.84M 17s
   100K .......... .......... .......... .......... ..........  0% 5.73M 14s
   150K .......... .......... .......... .......... ..........  0% 17.9M 11s
   200K .......... .......... .......... .......... ..........  0% 4.42M 11s
   250K .......... .......... .......... .......... ..........  0% 6.96M 10s
   300K .......... .......... .......... .......... ..........  0% 24.6M 9s
   350K .......... .......... .......... .......... .

  5800K .......... .......... .......... .......... .......... 13% 7.78M 3s
  5850K .......... .......... .......... .......... .......... 13% 37.2M 3s
  5900K .......... .......... .......... .......... .......... 13% 24.3M 3s
  5950K .......... .......... .......... .......... .......... 13% 38.0M 3s
  6000K .......... .......... .......... .......... .......... 13% 24.4M 3s
  6050K .......... .......... .......... .......... .......... 13% 10.1M 3s
  6100K .......... .......... .......... .......... .......... 13% 37.0M 3s
  6150K .......... .......... .......... .......... .......... 13% 5.23M 3s
  6200K .......... .......... .......... .......... .......... 14% 24.2M 3s
  6250K .......... .......... .......... .......... .......... 14% 41.4M 3s
  6300K .......... .......... .......... .......... .......... 14% 45.8M 3s
  6350K .......... .......... .......... .......... .......... 14% 9.71M 3s
  6400K .......... .......... .......... .......... .......... 14% 29.4M 3s
  6450K ....

 13250K .......... .......... .......... .......... .......... 29% 1.43M 3s
 13300K .......... .......... .......... .......... .......... 30% 30.1M 3s
 13350K .......... .......... .......... .......... .......... 30% 3.51M 3s
 13400K .......... .......... .......... .......... .......... 30% 35.7M 3s
 13450K .......... .......... .......... .......... .......... 30% 34.5M 3s
 13500K .......... .......... .......... .......... .......... 30% 38.2M 3s
 13550K .......... .......... .......... .......... .......... 30% 1.56M 3s
 13600K .......... .......... .......... .......... .......... 30% 29.4M 3s
 13650K .......... .......... .......... .......... .......... 30% 25.2M 3s
 13700K .......... .......... .......... .......... .......... 30% 41.4M 3s
 13750K .......... .......... .......... .......... .......... 31% 48.3M 3s
 13800K .......... .......... .......... .......... .......... 31% 37.2M 3s
 13850K .......... .......... .......... .......... .......... 31% 33.0M 3s
 13900K ....

 19950K .......... .......... .......... .......... .......... 45% 3.05M 2s
 20000K .......... .......... .......... .......... .......... 45% 3.39M 2s
 20050K .......... .......... .......... .......... .......... 45% 29.7M 2s
 20100K .......... .......... .......... .......... .......... 45% 32.6M 2s
 20150K .......... .......... .......... .......... .......... 45% 32.8M 2s
 20200K .......... .......... .......... .......... .......... 45% 11.6M 2s
 20250K .......... .......... .......... .......... .......... 45% 30.6M 2s
 20300K .......... .......... .......... .......... .......... 45% 18.8M 2s
 20350K .......... .......... .......... .......... .......... 45% 21.0M 2s
 20400K .......... .......... .......... .......... .......... 46% 1.88M 2s
 20450K .......... .......... .......... .......... .......... 46% 27.5M 2s
 20500K .......... .......... .......... .......... .......... 46% 34.9M 2s
 20550K .......... .......... .......... .......... .......... 46% 4.54M 2s
 20600K ....

 25900K .......... .......... .......... .......... .......... 58% 4.37M 2s
 25950K .......... .......... .......... .......... .......... 58% 38.8M 2s
 26000K .......... .......... .......... .......... .......... 58% 12.1M 2s
 26050K .......... .......... .......... .......... .......... 58% 15.5M 2s
 26100K .......... .......... .......... .......... .......... 58% 17.9M 2s
 26150K .......... .......... .......... .......... .......... 58% 28.0M 2s
 26200K .......... .......... .......... .......... .......... 59% 2.29M 2s
 26250K .......... .......... .......... .......... .......... 59% 5.41M 2s
 26300K .......... .......... .......... .......... .......... 59% 3.90M 2s
 26350K .......... .......... .......... .......... .......... 59% 27.1M 2s
 26400K .......... .......... .......... .......... .......... 59% 43.7M 2s
 26450K .......... .......... .......... .......... .......... 59% 44.9M 2s
 26500K .......... .......... .......... .......... .......... 59% 5.92M 2s
 26550K ....

 32250K .......... .......... .......... .......... .......... 72% 15.7M 1s
 32300K .......... .......... .......... .......... .......... 72% 20.4M 1s
 32350K .......... .......... .......... .......... .......... 72% 12.8M 1s
 32400K .......... .......... .......... .......... .......... 73% 42.1M 1s
 32450K .......... .......... .......... .......... .......... 73% 11.6M 1s
 32500K .......... .......... .......... .......... .......... 73% 5.47M 1s
 32550K .......... .......... .......... .......... .......... 73% 25.0M 1s
 32600K .......... .......... .......... .......... .......... 73% 42.2M 1s
 32650K .......... .......... .......... .......... .......... 73% 6.69M 1s
 32700K .......... .......... .......... .......... .......... 73% 34.4M 1s
 32750K .......... .......... .......... .......... .......... 73% 37.7M 1s
 32800K .......... .......... .......... .......... .......... 73% 28.9M 1s
 32850K .......... .......... .......... .......... .......... 74% 37.9M 1s
 32900K ....

 39350K .......... .......... .......... .......... .......... 88% 47.2M 0s
 39400K .......... .......... .......... .......... .......... 88% 6.18M 0s
 39450K .......... .......... .......... .......... .......... 88% 32.6M 0s
 39500K .......... .......... .......... .......... .......... 88% 6.37M 0s
 39550K .......... .......... .......... .......... .......... 89% 27.3M 0s
 39600K .......... .......... .......... .......... .......... 89% 31.5M 0s
 39650K .......... .......... .......... .......... .......... 89% 24.9M 0s
 39700K .......... .......... .......... .......... .......... 89% 7.95M 0s
 39750K .......... .......... .......... .......... .......... 89% 29.3M 0s
 39800K .......... .......... .......... .......... .......... 89% 41.2M 0s
 39850K .......... .......... .......... .......... .......... 89% 6.71M 0s
 39900K .......... .......... .......... .......... .......... 89% 4.92M 0s
 39950K .......... .......... .......... .......... .......... 90% 32.1M 0s
 40000K ....

In [7]:
model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
print(state_dict)

model.load_state_dict(state_dict)
model.eval()

{'conv1.weight': tensor([[[[-3.3291e-03,  5.3372e-03, -5.7548e-03,  ...,  1.3048e-02,
            2.5220e-02,  1.2310e-02],
          [ 2.8544e-03, -1.2225e-02, -2.1800e-02,  ...,  1.0162e-02,
            3.0931e-02,  1.8499e-02],
          [-4.0478e-03, -3.0228e-02, -4.3812e-02,  ..., -1.3756e-02,
            4.7455e-03,  2.2590e-03],
          ...,
          [ 1.8111e-02, -1.1865e-02, -2.7846e-02,  ..., -1.8882e-02,
           -9.7590e-03, -4.2343e-03],
          [ 3.0166e-02,  7.7134e-03, -1.3941e-02,  ..., -1.1360e-02,
            2.1114e-03,  4.3996e-03],
          [ 3.0019e-02,  1.5481e-02, -4.0438e-03,  ..., -3.4851e-03,
            7.3164e-03,  3.7749e-03]],

         [[ 1.7912e-03,  3.6975e-03, -9.5706e-03,  ...,  1.4056e-02,
            3.5389e-02,  3.4075e-02],
          [-1.2899e-03, -2.4197e-02, -3.6397e-02,  ...,  1.1081e-03,
            3.0996e-02,  3.1533e-02],
          [-3.7468e-03, -3.8118e-02, -5.4922e-02,  ..., -2.0059e-02,
            7.6029e-03,  1.7985e-02],
   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# load the image transformer
#trn = torchvision transforms
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)
print(classes)

('airfield', 'airplane_cabin', 'airport_terminal', 'alcove', 'alley', 'amphitheater', 'amusement_arcade', 'amusement_park', 'apartment_building/outdoor', 'aquarium', 'aqueduct', 'arcade', 'arch', 'archaelogical_excavation', 'archive', 'arena/hockey', 'arena/performance', 'arena/rodeo', 'army_base', 'art_gallery', 'art_school', 'art_studio', 'artists_loft', 'assembly_line', 'athletic_field/outdoor', 'atrium/public', 'attic', 'auditorium', 'auto_factory', 'auto_showroom', 'badlands', 'bakery/shop', 'balcony/exterior', 'balcony/interior', 'ball_pit', 'ballroom', 'bamboo_forest', 'bank_vault', 'banquet_hall', 'bar', 'barn', 'barndoor', 'baseball_field', 'basement', 'basketball_court/indoor', 'bathroom', 'bazaar/indoor', 'bazaar/outdoor', 'beach', 'beach_house', 'beauty_salon', 'bedchamber', 'bedroom', 'beer_garden', 'beer_hall', 'berth', 'biology_laboratory', 'boardwalk', 'boat_deck', 'boathouse', 'bookstore', 'booth/indoor', 'botanical_garden', 'bow_window/indoor', 'bowling_alley', 'boxin

In [10]:
# load the test image
img_name = '12.jpg'
if not os.access(img_name, os.W_OK):
    print('Fetching image')
    img_url = 'http://places.csail.mit.edu/demo/' + img_name
    os.system('wget ' + img_url)

Fetching image


--2022-03-21 22:07:13--  http://places.csail.mit.edu/demo/12.jpg
Resolving places.csail.mit.edu (places.csail.mit.edu)... 128.30.195.26, 128.30.195.26
Connecting to places.csail.mit.edu (places.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63736 (62K) [image/jpeg]
Saving to: ‘12.jpg’

     0K .......... .......... .......... .......... .......... 80%  658K 0s
    50K .......... ..                                         100% 26.4M=0.08s

2022-03-21 22:07:13 (814 KB/s) - ‘12.jpg’ saved [63736/63736]



In [11]:
img = Image.open(img_name)
input_img = V(centre_crop(img).unsqueeze(0))
input_img

tensor([[[[ 1.9407,  2.0605,  2.0263,  ..., -1.5528, -1.5185, -1.3302],
          [ 2.0434,  2.0605,  2.0092,  ..., -1.5528, -1.5185, -1.3130],
          [ 1.4612,  1.5982,  1.7523,  ..., -1.5699, -1.5357, -1.3302],
          ...,
          [-1.8439, -1.7069, -1.5699,  ..., -0.0972, -0.0972, -0.2171],
          [-1.9124, -1.8610, -1.6727,  ..., -0.1143, -0.0629, -0.1314],
          [-1.8610, -1.8610, -1.6213,  ..., -0.0287, -0.0801, -0.0458]],

         [[ 2.2010,  2.2710,  2.2185,  ..., -1.4405, -1.3704, -1.1604],
          [ 2.3060,  2.2710,  2.1835,  ..., -1.4405, -1.3880, -1.1779],
          [ 1.7283,  1.7983,  1.9209,  ..., -1.4580, -1.4055, -1.1779],
          ...,
          [-1.3179, -1.1779, -1.0203,  ..., -0.1625, -0.1625, -0.2850],
          [-1.3880, -1.3354, -1.1253,  ..., -0.1800, -0.1275, -0.1975],
          [-1.3529, -1.3354, -1.0728,  ..., -0.0924, -0.1450, -0.1099]],

         [[ 2.2566,  2.3960,  2.3786,  ..., -1.1944, -1.1944, -1.0898],
          [ 2.3088,  2.3786,  

In [12]:
print(input_img.size())

torch.Size([1, 3, 224, 224])


In [13]:
img = Image.open(img_name)
input_img = V(centre_crop(img).unsqueeze(0))

# forward pass
logit = model.forward(input_img)
h_x = F.softmax(logit, 1).data.squeeze()
probs, idx = h_x.sort(0, True)

print('{} prediction on {}'.format(arch,img_name))
# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

resnet18 prediction on 12.jpg
0.621 -> patio
0.296 -> restaurant_patio
0.021 -> porch
0.018 -> beer_garden
0.012 -> courtyard


In [1]:
!python run_placesCNN_basic.py

resnet18 prediction on 12.jpg
0.621 -> patio
0.296 -> restaurant_patio
0.021 -> porch
0.018 -> beer_garden
0.012 -> courtyard


In [15]:
!python run_placesCNN_basic.py --model_type 'resnet18' --folder '/Users/Administrator/DS/places365/images'

. <class 'str'>
opt:  Namespace(folder='/Users/Administrator/DS/places365/images', model_type='resnet18')
/Users/Administrator/DS/places365/images <class 'str'>
6
Prediction on /Users/Administrator/DS/places365/images/kitchen1.jpg
0.104 -> kitchen
0.084 -> clean_room
0.064 -> chemistry_lab
0.062 -> server_room
0.050 -> conference_room
Prediction on /Users/Administrator/DS/places365/images/office_cubicles.jpg
0.541 -> office_cubicles
0.164 -> office
0.058 -> clean_room
0.043 -> computer_room
0.042 -> conference_room
Prediction on /Users/Administrator/DS/places365/images/office_cubicles2.jpg
0.994 -> office_cubicles
0.006 -> office
0.000 -> server_room
0.000 -> computer_room
0.000 -> locker_room
Prediction on /Users/Administrator/DS/places365/images/office_cubicles3.jpg
0.393 -> office_cubicles
0.359 -> office
0.129 -> home_office
0.032 -> reception
0.020 -> computer_room
Prediction on /Users/Administrator/DS/places365/images/parking_lot1.jpg
0.998 -> parking_garage/indoor
0.001 -> parki

In [22]:
!python run_placesCNN_basic.py --model_type 'resnet18' --folder '/Users/Administrator/DS/places365/images'

opt:  Namespace(folder='/Users/Administrator/DS/places365/images', model_type='resnet18')
/Users/Administrator/DS/places365/images <class 'str'>
                   pred0     prob0  ...               pred4     prob4
0                kitchen  0.103796  ...     conference_room  0.050199
1        office_cubicles  0.540965  ...     conference_room  0.041712
2        office_cubicles  0.994000  ...         locker_room  0.000015
3        office_cubicles  0.392796  ...       computer_room  0.020245
4  parking_garage/indoor  0.997865  ...        auto_factory  0.000068
5  parking_garage/indoor  0.958753  ...  bus_station/indoor  0.001078

[6 rows x 10 columns]


In [ ]:
### Randomly sampling n images from folders




In [26]:
!pip install spacy nltk gensim textblob bokeh imutils pytesseract plotly hdbscan statsmodels

     |████████████████████████████████| 24.1 MB 10.9 MB/s eta 0:00:01   |████████████                    | 9.0 MB 7.5 MB/s eta 0:00:03     |███████████████████▋            | 14.8 MB 7.5 MB/s eta 0:00:02
     |████████████████████████████████| 636 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 18.5 MB 537 kB/s eta 0:00:011   |██                              | 1.2 MB 5.7 MB/s eta 0:00:04
     |████████████████████████████████| 27.7 MB 8.5 MB/s eta 0:00:01    |████                            | 3.4 MB 4.7 MB/s eta 0:00:06
     |████████████████████████████████| 5.2 MB 9.3 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 233 kB 10.0 MB/s eta 0:00:01
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-

In [30]:
#utils2.py package can be found under https://github.com/philippe-heitzmann/Object_Detection
import sys
sys.path.append('/Users/Administrator/DS/DeepRoad/Object_Detection')
sys.path.append('/Users/phil0/DS/DeepRoad/Object_Detection')
import utils2
from importlib import reload
reload(utils2)
from utils2 import *

results1 = get_files_in_dir('jpg', 'png', path = '/Users/Administrator/DS/places365/images', fullpath = True)
results1 = np.random.choice(results1, size=1, replace=False) #, p=[0.3, 0.5, 0.0, 0.2]
print(results1)

0.0 sec to complete <function get_files_in_dir at 0x7eff078ede50>
['/Users/Administrator/DS/places365/images/office_cubicles3.jpg']


In [44]:


allfiles = getListOfFiles('jpg', dirName = '/Users/Administrator/DS/places365/images')
print(allfiles, len(allfiles))

['/Users/Administrator/DS/places365/images/kitchen/kitchen1.jpg', '/Users/Administrator/DS/places365/images/office_cubicles/office_cubicles.jpg', '/Users/Administrator/DS/places365/images/office_cubicles/office_cubicles2.jpg', '/Users/Administrator/DS/places365/images/office_cubicles/office_cubicles3.jpg', '/Users/Administrator/DS/places365/images/parking_garage_indoor/parking_lot1.jpg', '/Users/Administrator/DS/places365/images/parking_garage_indoor/parking_lot2.jpg'] 6
